In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
import sys
from pathlib import Path

root = str(Path('~').expanduser())  # get your root path (C:\\Users\your_name)
maxlik_path = "\PycharmProjects\Econometrics-Course\maxlik"
sys.path.insert(0, root + maxlik_path)  # add MaxLik path in your system (sys) path list
from maxlik import MaxLik  # now you can import MaxLik class

In [12]:
def AR_1_log_like(pars_vect: np.array, data: np.array):
    T = data.shape[0]
    c = pars_vect[0]
    phi = pars_vect[1]
    sig2 = pars_vect[2]
    err = np.zeros((T,1))
    err[0] = data[0]-c
    for i in range(1,len(data)):
        err[i]=data[i]-c-phi*data[i-1]
    lik = np.zeros((T,1))
    for i in range(0,len(data)):
        lik[i] = -(1/2)*np.log(2*np.pi*sig2)-(err[i]**2)/(2*sig2)
    return lik

In [13]:
def MA_1_log_like(pars_vect, data):
    T = data.shape[0]
    c = pars_vect[0]
    psi = pars_vect[1]
    sig2 = pars_vect[2]
    err = np.zeros((T,1))
    err[0] = data[0]-c
    for i in range(1,len(data)):
        err[i] = data[i]-c-psi*err[i-1]
    lik = np.zeros((T,1))
    for i in range(0,len(data)):
        lik[i] = -(1/2)*np.log(2*np.pi*sig2)-(err[i]**2)/2*sig2
    return lik

In [6]:
data: pd.DataFrame = pd.read_csv("~/PycharmProjects/Econometrics-Course/Dataset/GDP.csv", index_col="DATE")
data.head()

,GDP
DATE,
2000-01-01,10002.857
2000-04-01,10247.679
2000-07-01,10319.825
2000-10-01,10439.025
2001-01-01,10472.879


In [7]:
log_gdp: pd.DataFrame = np.log(data).diff().dropna()
log_gdp.head()

,GDP
DATE,
2000-04-01,0.024180
2000-07-01,0.007016
2000-10-01,0.011484
2001-01-01,0.003238
2001-04-01,0.011860


In [8]:
log_gdp = log_gdp.to_numpy()
initial_guess = np.array([0, 0.8, 0.1])
bounds = [[-9999, 9999], [-0.9999, 0.9999], [0.0001, 9999]]

In [14]:
maxlik_ar = MaxLik(
    func_vector=AR_1_log_like,
    data=log_gdp,
    initial_guess=initial_guess,
    bounds=bounds,
    method_se_optimization="sandwich"
)
maxlik_ar.get_summary(printing=True)

maxlik_ma = MaxLik(
    func_vector=MA_1_log_like,
    data=log_gdp,
    initial_guess=initial_guess,
    bounds=bounds,
    method_se_optimization="sandwich"
)
maxlik_ma.get_summary(printing=True)

======================== MAX LIK OPTIMIZER - by Prof.Carlini et Al. ===============================

function minimized: AR_1_log_like
elapsed time: 0.251296s

number of observation: 83
number of parameters: 3
function value: [-2.74821933]

variance-covariance method: sandwich


       beta value     SE  Z-test conf p-value
beta0      0.011  0.004   2.839  ***   0.005
beta1    -0.1975  0.292  -0.676        0.499
beta2     0.0002    0.0   1.637    *   0.102

 * alpha = 0.1
 ** alpha = 0.05
 *** alpha = 0.01

================================= ENJOY ECONOMETRICS GUYS =======================================
======================== MAX LIK OPTIMIZER - by Prof.Carlini et Al. ===============================

function minimized: MA_1_log_like
elapsed time: 0.074334s

number of observation: 83
number of parameters: 3
function value: [-3.68623162]

variance-covariance method: sandwich


       beta value     SE   Z-test conf p-value
beta0     0.0003  0.003    0.106        0.916
beta1     0.7999

,beta value,SE,Z-test,p-value
beta0,0.000295,0.002786,0.105974,0.915603
beta1,0.799950,0.004738,168.843473,0.000000
beta2,0.000100,0.000010,9.606834,0.000000
